In [ ]:
#Case Study 1: Generative Text for Customer Support Automation
#Project Overview: Develop an AI-powered system to automate customer support interactions using generative models like GPT-3.5.
#Use Case:
#Automated Response Generation:
#Problem Statement: Customer support teams are overwhelmed by repetitive inquiries that could be handled by automated systems.
#Solution: Implement a generative AI model to automatically generate accurate and context-aware responses to common customer queries, reducing the load on human agents and improving response times.

In [2]:
!pip install transformers


     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     --------- ------------------------------ 10.2/43.6 kB ? eta -:--:--
     ----------------- -------------------- 20.5/43.6 kB 162.5 kB/s eta 0:00:01
     ----------------------------------- -- 41.0/43.6 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 43.6/43.6 kB 212.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.3 MB 1.3 MB/s eta 0:00:08
   ---------------------------------------- 0.0/9.3 MB 487.6 kB/s eta 0:00:20
   ---------------------------------------- 0.1/9.3 MB 744.7 kB/s eta 0:00:13
   ---------------------------------------- 0.1/9.3 MB 726.2 kB/s eta 0:00:13
    --------------------------------------- 0.2/9.3 MB 980.4 kB/s eta 0:00:10
    --------------------------------------- 0.2/9.3 MB 1.1 MB/s eta 0:00:09
   - -------------------------------------- 0.3/9.3 MB 1.1 MB/s eta 0:00:09
   - -

In [2]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

file_path = r'C:\Users\Nirmiti\customer_support_data.csv'
data = pd.read_csv(file_path)

# Loading pre-trained tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Tokenize and encoding text data
queries = data['query'].tolist()
responses = data['response'].tolist()

# Tokenize and encoding queries and responses
encoded_queries = [tokenizer.encode(query, add_special_tokens=True) for query in queries]
encoded_responses = [tokenizer.encode(response, add_special_tokens=True) for response in responses]

# Displaying the first encoded query and response
print(encoded_queries[0])
print(encoded_responses[0])


[3666, 598, 318, 21899]
[23433, 6865, 301, 9221, 262, 598, 290, 4155, 534, 3335, 338, 3788, 318, 510, 284, 3128, 13]


In [3]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

tokenizer.pad_token = tokenizer.eos_token

from torch.utils.data import Dataset, DataLoader

class CustomerSupportDataset(Dataset):
    def __init__(self, queries, responses, tokenizer, max_length=512):
        self.queries = queries
        self.responses = responses
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.queries)

    def __getitem__(self, idx):
        query = self.queries[idx]
        response = self.responses[idx]
        
        # Encoding the query and response together with a separator token
        encoding = self.tokenizer.encode_plus(
            query,
            response,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt',
            return_attention_mask=True
        )

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': input_ids
        }

# Creating the dataset and dataloader
dataset = CustomerSupportDataset(encoded_queries, encoded_responses, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Displaying a batch of data
for batch in dataloader:
    print(batch)
    break


{'input_ids': tensor([[ 2437,   466,   314,  ..., 50256, 50256, 50256],
        [   40,  4398,   470,  ..., 50256, 50256, 50256],
        [ 6090,   314, 12233,  ..., 50256, 50256, 50256],
        ...,
        [ 3666,  1502,   318,  ..., 50256, 50256, 50256],
        [ 3666,   598,   318,  ..., 50256, 50256, 50256],
        [ 6090,   314, 12233,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 2437,   466,   314,  ..., 50256, 50256, 50256],
        [   40,  4398,   470,  ..., 50256, 50256, 50256],
        [ 6090,   314, 12233,  ..., 50256, 50256, 50256],
        ...,
        [ 3666,  1502,   318,  ..., 50256, 50256, 50256],
        [ 3666,   598,   318,  ..., 50256, 50256, 50256],
        [ 6090,   314, 12233,  ..., 50256, 50256, 50256]])}


In [ ]:
from transformers import GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import re
import torch
from torch.utils.data import Dataset, DataLoader

# Loading the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Moving the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Defining the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(dataloader) * 3  # 3 epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0, 
                                            num_training_steps=total_steps)

# Defining the training loop
def train(model, dataloader, optimizer, scheduler, device, epochs=3):
    model.train()
    
    for epoch in range(epochs):
        for batch in dataloader:
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            scheduler.step()

            print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

# Training the model
train(model, dataloader, optimizer, scheduler, device)


C:\Users\Nirmiti\anaconda3\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            total_loss += loss.item()
    
    return total_loss / len(dataloader)

# Evaluating the model
validation_loss = evaluate(model, dataloader, device)
print(f'Validation Loss: {validation_loss}')


In [ ]:
model.save_pretrained('C:\Users\Nirmiti\save\model')
tokenizer.save_pretrained('C:\Users\Nirmiti\save\tokenizer')


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Loading the saved model and tokenizer
model = GPT2LMHeadModel.from_pretrained('C:\Users\Nirmiti\save\model')
tokenizer = GPT2Tokenizer.from_pretrained('C:\Users\Nirmiti\save\tokenizer')

# Function to generate response
def generate_response(query):
    inputs = tokenizer.encode(query, return_tensors='pt')
    outputs = model.generate(inputs, max_length=150, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example usage
query = "How do I reset my password?"
response = generate_response(query)
print(response)